# 코드 설명
- 신용 대출 신청 여부 예측 전처리 
- user_spec, loan_result 전처리한 데이터 생성 후 저장하는 코드

# 1. Library, Data

In [ ]:
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
from tqdm import tqdm
from sklearn.impute import KNNImputer

In [ ]:
os.chdir(r'C:\Users\styli\Desktop\빅콘 최종 돌리기 LGBM') #상대 경로 지정
user_spec = pd.read_csv('user_spec.csv')
user_spec = user_spec.sort_values(['user_id','insert_time']) # 데이터 정렬 
loan_result = pd.read_csv('loan_result.csv')

# 2. user_spec 전처리

In [ ]:
def f_personal_rehabilitation(x):
  if x == '1.0/0.0': return 1
  elif x == '1.0/1.0': return 2
  else: return 0

def preprocessing_user_spec(user_spec):
  user_spec = user_spec.sort_values(['user_id','insert_time']) # 데이터 정렬

  '''
  개인회생(personal_rehabilitation) 변수 생성
  개인회생자가 아닌 경우 : (0,0), (0,Nan), (Nan,Nan), (0,1) -> 0
  개인회생자인데 납입중 : (1,0) -> 1
  개인회생자인데 납입 완료 : (1,1) -> 2
  '''

  user_spec['personal_rehabilitation'] = user_spec['personal_rehabilitation_yn'].astype(str) + '/' +user_spec['personal_rehabilitation_complete_yn'].astype(str)
  user_spec['personal_rehabilitation'] = user_spec['personal_rehabilitation'].apply(f_personal_rehabilitation)
  user_spec.drop(columns=['personal_rehabilitation_yn','personal_rehabilitation_complete_yn'], inplace=True) # 컬럼 제거

  '''
  기대출수, 기대출수 결측치 보간
  (1, nan)인 경우 : (1, 기대출 수가 1인 경우의 기대출금액의 중앙값)으로 채움
  (nan, nan)인 경우 : 대출이 없다고 가정하여 (0, 0)으로 채움
  '''

  user_spec['existing_loan_cnt'] = user_spec['existing_loan_cnt'].fillna(0)
  user_spec['existing_loan_amt'] = user_spec['existing_loan_amt'].fillna('None')

  col_list = []

  for i in range(len(user_spec)):
    existing_loan_cnt = user_spec.existing_loan_cnt.iloc[i]
    existing_loan_amt = user_spec.existing_loan_amt.iloc[i]
    
    if existing_loan_amt == 'None':

      if existing_loan_cnt == 0:
        col_list.append(0)

      elif existing_loan_cnt == 1:
        col_list.append(10000000)

    else:
      col_list.append(existing_loan_amt)

  user_spec['existing_loan_amt'] = col_list

  '''
  user_id 별 이전값으로 보간
  '''

  ndf = []
  for idx, df in tqdm(user_spec.groupby('user_id')):
    ndf.append(df.fillna(method='ffill'))

  user_spec = pd.concat(ndf, axis = 0, ignore_index=True)

  '''
  user_id 별 이후값으로 보간
  '''

  ndf = []
  for idx, df in tqdm(user_spec.groupby('user_id')):
    ndf.append(df.fillna(method='bfill'))

  user_spec = pd.concat(ndf, axis = 0, ignore_index=True)

  '''
  yearly_income 결측치 제거
  '''

  user_spec = user_spec.dropna(subset=['yearly_income']) # yearly_income 결측값인 행 제거 
  user_spec.reset_index(drop=True, inplace=True) # 인덱스 초기화

  '''
  KNN 결측치 보간
  '''

  col_names = ['birth_year', 'gender', 'credit_score', 'yearly_income', 
             'income_type', 'company_enter_month', 'employment_type', 'houseown_type', 
             'desired_amount', 'purpose', 'existing_loan_cnt', 'existing_loan_amt', 
             'personal_rehabilitation']

  knn_data = user_spec[col_names] # knn data 생성 

  # label encoding
  label_col = knn_data.select_dtypes('object').columns
  label_dict = {}

  for i in label_col:
      label_dict[i] = {val : idx for idx, val in enumerate(knn_data[i].unique())}
      knn_data[i] = knn_data[i].map(label_dict[i])

  # knn
  imputer = KNNImputer(n_neighbors=1)
  knn_data = imputer.fit_transform(knn_data)

  # 데이터프레임으로 변환
  knn_data = pd.DataFrame(knn_data)
  knn_data.columns = col_names

  #역변환
  for i in label_col:
      inv_map = {v: k for k, v in label_dict[i].items()}
      knn_data[i] = knn_data[i].map(inv_map)
      
  # 제거했던 컬럼 추가
  knn_data['application_id'] = user_spec['application_id']
  knn_data['user_id'] = user_spec['user_id']
  knn_data['insert_time'] = user_spec['insert_time']

  return knn_data

In [ ]:
user_spec = preprocessing_user_spec(user_spec) # user_spec 전처리 

In [ ]:
user_spec.to_csv('user_spec_filled_final.csv', index=False) # 전처리 완료한 user_spec csv 파일 저장

# 3. loan_result 전처리 

In [ ]:
def preprocessing_loan_result(loan_result, user_spec):
    '''
    loan_rate 결측치 처리: 같은 bank id, product id의 loan_rate 평균으로 보간 
    '''
    loan_result['bank_product'] = loan_result['bank_id'].astype('str')+' '+loan_result['product_id'].astype('str') # 전처리용 bank_product 변수 추가

    ndf = []

    for idx, df in tqdm(loan_result.groupby('bank_product')):
        df['loan_rate'] = df['loan_rate'].fillna(df['loan_rate'].mean())
        ndf.append(df)
      
    loan_result = pd.concat(ndf,axis=0) # data 결합
    loan_result = loan_result.sort_index() # index 정렬
    loan_result.drop(columns=['bank_product'], inplace=True) # 전처리용 bank_product 컬럼 제거
   
    '''
    loan_rate 결측치 처리: 같은 application_id의 loan_rate 최소값으로 보간 
    '''
    ndf = []

    for idx, df in tqdm(loan_result.groupby('application_id')):
        df['loan_rate'] = df['loan_rate'].fillna(df['loan_rate'].min())
        ndf.append(df)

    loan_result = pd.concat(ndf,axis=0) # data 결합
    loan_result = loan_result.sort_index() # index 정렬

    '''
    loan_limit 결측치 처리: desired_amount로 결측값 보간
    '''
    user_desired_amount = user_spec[['application_id','desired_amount']]
    loan_result = pd.merge(loan_result, user_desired_amount, how='left',on='application_id')

    ndf = []

    for idx, df in tqdm(loan_result.groupby('application_id')):
        df['loan_limit'] = df['loan_limit'].fillna(df['desired_amount'])
        ndf.append(df)
                  
    loan_result = pd.concat(ndf,axis=0) # data 결합
    loan_result = loan_result.sort_index() # index 정렬
    loan_result.drop(columns=['desired_amount'], inplace=True) # 전처리용 desired_amount 컬럼 제거
    
    '''
    그 외 loan_rate, loan_limit 결측치 제거
    '''
    loan_result.dropna(subset=['loan_limit','loan_rate'], inplace = True)
    loan_result.reset_index(drop=True, inplace = True) # index 초기화
    
    return loan_result

In [ ]:
loan_result = preprocessing_loan_result(loan_result, user_spec) # loan_result 전처리 

In [ ]:
loan_result.to_csv('loan_result_fillna.csv', index=False) # 전처리 완료한 loan_result csv 파일 저장